In [2]:
exec('from __future__ import unicode_literals')

import os
import sys
import random

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

### Load a paragraph from 10-K docs (assume current directory is OpenNMT-kpg/notebook/)

In [3]:
data_root_path = '/export/share/rmeng/project/OpenNMT-kpg/data/10k'
data_root_path = '../data/10k'
print(os.path.abspath(data_root_path))
doc_dicts = []
for subdir, dirs, files in os.walk(data_root_path):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
#             print(filepath)
            text = open(filepath, 'r').readlines()
            pars = [l for l in text if len(l.split()) > 20]
            doc = {'name': file, 'path': filepath, 'text': text, 'paragraph': pars}
#             print('#line=%d, #par=%d' % (len(text), len(pars)))
            doc_dicts.append(doc)
    
print('Loaded #(10k docs)=%d' % (len(doc_dicts)))

/Users/rui.meng/project/OpenNMT-kpg/data/10k
Loaded #(10k docs)=0


##### Sample a paragraph

In [12]:
doc_id = random.randint(0, len(doc_dicts))
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par_id = random.randint(0, len(doc_dicts[doc_id]['paragraph']))
text_to_extract = pars[par_id]
print(doc_id)
print(par_id)
print(text_to_extract)

7705
3
CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.



In [11]:
# doc_id = 9539
# par_id = 67
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par = pars[par_id]
print(doc_id)
print(par_id)
print(par)

IndexError: list index out of range

### Or load a PROD description

In [ ]:
PROD_path = '/export/share/rmeng/project/OpenNMT-kpg/data/SF_Prod'

prod_dicts = []
for subdir, dirs, files in os.walk(PROD_path):
    for file in files:
        filepath = subdir + os.sep + file
        text = open(filepath, 'r').readlines()
        text = '\n'.join([l.strip() for l in text])
        doc = {'name': file, 'path': filepath, 'text': text}
        prod_dicts.append(doc)

print('Loaded #(PROD docs)=%d' % (len(prod_dicts)))

In [ ]:
doc_id = random.randint(0, len(prod_dicts))
doc = prod_dicts[doc_id]
text_to_extract = doc['text']
print(doc_id)
print(doc['name'])
print(text_to_extract)

### Supervised Deep Keyphrase Model

In [7]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = build_translator(opt, report_score=False)

In [15]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


Translating 10/1
Total translation time (s): 0.421937
Average translation time (s): 0.421937
Tokens per second: 4.740044


In [16]:
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

Paragraph:
	CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.

Top predictions:
	1: coal fields
	2: CSX
	3: 1978
	4: Virginia
	5: Company
	6: Chessie
	7: merger
	8: New
	9: CSX.
	10: freight transportation
	11: law.
	12: rail network
	13: Seaboard
	14: co

### PKE models

##### Initialize DF (only run once)

In [ ]:
from pke import compute_document_frequency
from string import punctuation

# stoplist for filtering n-grams
stoplist=list(punctuation)

# compute df counts and store as n-stem -> weight values
compute_document_frequency(input_dir=data_root_path,
                           output_file=data_root_path+'/10k.df.tsv.gz',
                           extension='xml',           # input file extension
                           language='en',                # language of files
                           normalization="stemming",    # use porter stemmer
                           stoplist=stoplist)

#### TF-IDF

In [17]:
import string
import pke

# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. select {1-3}-grams not containing punctuation marks as candidates.
extractor.candidate_selection(n=3, stoplist=list(string.punctuation))

# 4. weight the candidates using a `tf` x `idf`
df = pke.load_document_frequency_file(input_file=data_root_path+'/10k.df.tsv.gz')
extractor.candidate_weighting(df=df)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


	1: csx (0.0000)
	2: csx was (0.0000)
	3: csx was incorporated (0.0000)
	4: was (0.0000)
	5: was incorporated (0.0000)
	6: was incorporated in (0.0000)
	7: incorporated (0.0000)
	8: incorporated in (0.0000)
	9: incorporated in 1978 (0.0000)
	10: in 1978 (0.0000)


#### YAKE

In [18]:
import pke
from nltk.corpus import stopwords

# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3, stoplist=stoplist)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              stoplist=stoplist,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: seaboard coast line (0.0068)
	2: coast line industries (0.0068)
	3: virginia law (0.0073)
	4: chessie system (0.0348)
	5: seaboard coast (0.0348)
	6: coast line (0.0348)
	7: line industries (0.0348)
	8: company (0.0410)
	9: including new england (0.0450)
	10: new york metropolitan (0.0496)


#### TextRank

In [40]:
import pke

# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: our data practices (0.1481)
	2: data protection laws (0.1409)
	3: data protection (0.1090)
	4: our business practices (0.0960)
	5: data (0.0796)
	6: manner adverse (0.0750)
	7: regulatory proposals (0.0750)
	8: our business (0.0685)
	9: substantial costs (0.0578)
	10: such interpretations (0.0578)
